# Item Collaborative Filtering
* This notebook implements item-based collaborative filtering
* Prediction is $\tilde r_{ij} = \dfrac{\sum_{k \in N(j)} w_{kj}^{\lambda_w}r_{ik}^{\lambda_r}}{\sum_{k \in N(j)} w_{kj}^{\lambda_w} + \lambda}$ for item-based collaborative filtering
* $r_{ij}$ is the rating for user $i$ and item $j$
* $w_{kj}$ is the similarity between items $j$ and $k$
* $N(j)$ is the largest $K$ sorted by $w_{kj}$
* $\lambda_w, \lambda_r, \lambda$ are regularization parameters

In [ ]:
residual_alphas = [];
@nbinclude("Alpha.ipynb");

## Determine the neighborhoods for each user and item

In [ ]:
function read_similarity_matrix(outdir)
    read_params(outdir)["S"]
end;

In [ ]:
function get_abs_neighborhood(item, S, K)
    weights = S[:, item]
    # ensure that the neighborhood for an item does not include itself
    weights[item] = Inf
    K = Int(min(K, length(weights) - 1))    
    order = partialsortperm(abs.(weights), 2:K+1, rev = true)
    order, weights[order]
end;

In [ ]:
isnonzero(x) = !isapprox(x, 0.0, atol=eps(Float64))

# exponentially decay x
function decay(x, a)
    isnonzero(x) ? sign(x) * abs(x)^a : zero(eltype(a))
end

# each prediction is just the weighted sum of all items in the neighborhood
# we apply regularization terms to decay the weights, ratings, and final prediction
function make_prediction(item, users, R, get_neighborhood, λ)
    if item > size(R)[2]
        # the item was not in our training set; we have no information
        return zeros(length(item))
    end
    items, weights = get_neighborhood(item)
    weights = decay.(weights, λ[1])
    predictions = zeros(eltype(weights), length(users))
    weight_sum = zeros(eltype(weights), length(users))
    for u = 1:length(users)
        for (i, weight) in zip(items, weights)
            if isnonzero(R[users[u], i])
                predictions[u] += weight * decay(R[users[u], i], λ[2])
                weight_sum[u] += abs(weight)
            end
        end
    end
    for u = 1:length(users)
        if isnonzero(weight_sum[u] + λ[3])
            predictions[u] /= (weight_sum[u] + λ[3])
        end
    end
    predictions
end;

In [ ]:
function collaborative_filtering(training, inference, get_neighborhood, λ)
    R = sparse(
        training.user,
        training.item,
        training.rating,
        maximum(training.user),
        maximum(training.item),
    )

    preds = zeros(eltype(λ), length(inference.rating))
    @tprogress Threads.@threads for item in collect(Set(inference.item))
        mask = inference.item .== item
        preds[mask] =
            make_prediction(item, inference.user[mask], R, get_neighborhood, λ)
    end

    preds
end;

In [ ]:
Base.@kwdef mutable struct cf_params
    name::Any
    training_residuals::Any
    validation_residuals::Any
    neighborhood_type::Any
    S::Any # the similarity matrix
    K::Any # the neighborhood size
    λ::Vector{Float64} = [1.0, 1.0, 0.0] # [weight_decay, rating_decay, prediction_decay]
end;

to_dict(x::T) where {T} = Dict(string(fn) => getfield(x, fn) for fn ∈ fieldnames(T));

## Item based CF

In [ ]:
function get_training(residual_alphas)
    get_residuals("training", residual_alphas)
end

function get_validation(residual_alphas)
    get_residuals("validation", residual_alphas)
end

function get_inference()
    training = get_split("training")
    validation = get_split("validation")
    test = get_split("test")
    RatingsDataset(
        user = [training.user; validation.user; test.user],
        item = [training.item; validation.item; test.item],
        rating = fill(
            0.0,
            length(training.rating) + length(validation.item) + length(test.item),
        ),
    )
end;

In [ ]:
function optimize_model(param)
    # unpack parameters
    training = get_training(param.training_residuals)
    validation = get_validation(param.validation_residuals)
    S = read_similarity_matrix(param.S)
    K = param.K
    neighborhood_types = Dict("abs" => get_abs_neighborhood)
    neighborhoods = i -> neighborhood_types[param.neighborhood_type](i, S, K)

    # optimize hyperparameters
    function validation_mse(λ)
        pred = collaborative_filtering(training, validation, neighborhoods, λ)
        truth = validation.rating
        β = pred \ truth
        loss = mse(truth, pred .* β)
        @debug "loss: $loss β: $β: λ $λ"
        loss
    end
    res = optimize(
        validation_mse,
        param.λ,
        LBFGS(),
        autodiff = :forward,
        Optim.Options(show_trace = true, extended_trace = true),
    )
    param.λ = Optim.minimizer(res)

    # save predictions
    inference = get_inference()
    preds = collaborative_filtering(training, inference, neighborhoods, param.λ)
    sparse_preds = sparse(inference.user, inference.item, preds)
    function model(users, items, predictions)
        result = zeros(length(users))
        for i = 1:length(users)
            if users[i] <= size(predictions)[1] && items[i] <= size(predictions)[2]
                result[i] = predictions[users[i], items[i]]
            end
        end
        result
    end
    write_predictions(
        (users, items) -> model(users, items, sparse_preds),
        outdir = param.name,
        residual_alphas = param.validation_residuals,
        save_training = true,
    )
    write_params(to_dict(param), outdir = param.name)
end;